In [1]:
import pandas as pd

In [2]:
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [3]:
prices.isna().sum()

product_id    0
old_price     0
new_price     0
updated_at    0
dtype: int64

In [4]:
sales.isna().sum()

product_id          0
ordered_at          0
quantity_ordered    0
dtype: int64

In [5]:
prices['updated_at'] = pd.to_datetime(prices['updated_at'], format='%m/%d/%y %H:%M')
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'], format='%m/%d/%y %H:%M')

In [6]:
combine = pd.merge_asof(sales.sort_values('ordered_at'), prices.sort_values('updated_at'), by='product_id', left_on='ordered_at', right_on='updated_at', direction='nearest')

In [7]:
combine.loc[combine['ordered_at'] >= combine['updated_at'], 'per_product_price'] = combine['new_price']
combine.loc[combine['ordered_at'] <  combine['updated_at'], 'per_product_price'] = combine['old_price']

In [8]:
combine['final_price'] = combine['per_product_price'] * combine['quantity_ordered']

In [9]:
combine.groupby(['product_id', 'per_product_price'], as_index=False)['final_price'].agg({'revenue':'sum'})

,product_id,per_product_price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [10]:
combine.groupby(['product_id'], as_index=False)['final_price'].agg({'revenue':'sum'})

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0
